In [ ]:
# default_exp g_models.generative_base

# Generative Base

> Base class for generative models

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.layers import *

In [ ]:
# export

class GenerativeModel(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        raise NotImplementedError
        
    def x_to_latent(self, x):
        raise NotImplementedError
        
    def sample(self, **sample_kwargs):
        raise NotImplementedError
        
    def sample_no_grad(self, **sample_kwargs):
        with torch.no_grad():
            return self.sample(**sample_kwargs)
        
    def get_rl_tensors(self):
        raise NotImplementedError

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()